### Dependencies

In [18]:
import pandas as pd
from pathlib import Path
import re, ast, numpy as np

### 1. Data Exploration/Preparation

In [8]:
df = pd.read_csv(Path("../data/external/kaggle_spotify_data.csv"))

In [9]:
df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [10]:
df.dtypes

Unnamed: 0            int64
track_id             object
artists              object
album_name           object
track_name           object
popularity            int64
duration_ms           int64
explicit               bool
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature        int64
track_genre          object
dtype: object

In [11]:
df = df.drop(columns=["Unnamed: 0"])

Removed column because it just repeated the index of the rows.

In [17]:
df.duplicated().sum()
dupes = df[df.duplicated(keep=False)]
dupes.head()

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
1874,0CDucx9lKxuCZplLXUz0iX,Buena Onda Reggae Club,Disco 2,Song for Rollins,16,219346,False,0.841,0.577,0,-7.544,1,0.0438,0.238000,0.860000,0.0571,0.843,90.522,4,afrobeat
1925,0CDucx9lKxuCZplLXUz0iX,Buena Onda Reggae Club,Disco 2,Song for Rollins,16,219346,False,0.841,0.577,0,-7.544,1,0.0438,0.238000,0.860000,0.0571,0.843,90.522,4,afrobeat
2109,2aibwv5hGXSgw7Yru8IYTO,Red Hot Chili Peppers,Stadium Arcadium,Snow (Hey Oh),80,334666,False,0.427,0.900,11,-3.674,1,0.0499,0.116000,0.000017,0.1190,0.599,104.655,4,alt-rock
2155,2aibwv5hGXSgw7Yru8IYTO,Red Hot Chili Peppers,Stadium Arcadium,Snow (Hey Oh),80,334666,False,0.427,0.900,11,-3.674,1,0.0499,0.116000,0.000017,0.1190,0.599,104.655,4,alt-rock
3683,7mULVp0DJrI2Nd6GesLvxn,Joy Division,Timeless Rock Hits,Love Will Tear Us Apart,0,204621,False,0.524,0.902,2,-8.662,1,0.0368,0.000989,0.695000,0.1370,0.907,146.833,4,alternative


In [22]:
df["track_name_clean"] = (df['track_name'].fillna("").astype(str).str.lower().str.replace(r"\s*\([^)]*\)\s*$", "", regex=True).str.replace(r"\s+", " ", regex=True).str.strip())

"Song (Live)" and "Song (Remastered 2012)" become just "song"

In [32]:
df[df['artists']=="Justin Bieber"]

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,track_name_clean
81118,6I3mqTwhRpn34SLVafSH7G,Justin Bieber,Justice,Ghost,88,153190,False,0.601,0.741,2,...,1,0.0478,0.1850,0.000029,0.4150,0.441,153.960,4,pop,ghost
81264,2TKYYJSJCYTn8BiSl66WY1,Justin Bieber,pov: it's 2021,Ghost,2,153190,False,0.604,0.741,2,...,1,0.0475,0.1780,0.000035,0.4090,0.473,153.947,4,pop,ghost
81267,1dqe1m4c2ynDGRctTV4Cj8,Justin Bieber,Die For You,We’re In This Together,0,181640,False,0.706,0.644,11,...,0,0.2970,0.1250,0.000000,0.1080,0.624,149.452,4,pop,we’re in this together
81268,0GPeyv1Oj1LbqyFJHM3j4C,Justin Bieber,Holidays Are Coming,Mistletoe,2,183066,False,0.623,0.668,6,...,0,0.0531,0.4750,0.000000,0.0862,0.823,161.948,4,pop,mistletoe
81269,5LseDWFft6Fhwj5iK4fD5E,Justin Bieber,Santa Claus Is Coming To Town,Mistletoe,0,183066,False,0.623,0.668,6,...,0,0.0531,0.4750,0.000000,0.0862,0.823,161.948,4,pop,mistletoe
81272,499GJHKeK9sZLfM03XejN0,Justin Bieber,Holiday Anthems 2022,Mistletoe,2,183066,False,0.623,0.668,6,...,0,0.0531,0.4750,0.000000,0.0862,0.823,161.948,4,pop,mistletoe
81273,3AkPOVgTS0WbLqoGQI90ZO,Justin Bieber,World's Best Christmas Playlist,Mistletoe,4,183066,False,0.623,0.668,6,...,0,0.0531,0.4750,0.000000,0.0862,0.823,161.948,4,pop,mistletoe
81275,3HqSKkoH7DqzfJYrBqvzSq,Justin Bieber,Pop Christmas 2022,Rockin' Around The Christmas Tree,0,132732,False,0.464,0.642,5,...,1,0.0320,0.0189,0.000000,0.0957,0.828,151.592,4,pop,rockin' around the christmas tree
81277,7wNeq4RKcnjjTbEF5K9qAR,Justin Bieber,JULMYS,Christmas Eve,1,223613,False,0.409,0.570,6,...,0,0.0711,0.6870,0.000000,0.1730,0.336,128.657,4,pop,christmas eve
81278,6ovqCJRzKl9JxdR0pR5We4,Justin Bieber,JULMYS,Mistletoe,0,183066,False,0.623,0.668,6,...,0,0.0531,0.4750,0.000000,0.0862,0.823,161.948,4,pop,mistletoe
